In [19]:
import torch
import torchvision.models as models

device = torch.device("mps")

# Load pre-trained VGG16
vgg16 = models.vgg16(pretrained=True)

# Freeze all layers (optional, but recommended for fine-tuning)
for param in vgg16.parameters():
    param.requires_grad = False

/Users/boss/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/boss/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [20]:
import torch.nn as nn

# Modify the classifier (final layers)
vgg16.classifier = nn.Sequential(
    nn.Linear(25088, 4096),  # VGG16's original FC layer
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5),
    nn.Linear(4096, 4096),   # VGG16's original FC layer
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5),
    nn.Linear(4096, 7)      # New output layer for 9 classes
).to(device)

In [21]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to match VGG16 input size
    transforms.ToTensor(),          # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

# Load dataset
train_dataset = datasets.ImageFolder(root='../data/train', transform=transform)
val_dataset = datasets.ImageFolder(root='../data/test', transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [22]:
import torch.optim as optim

# Define loss function
criterion = nn.CrossEntropyLoss()

# Define optimizer (only train the classifier parameters)
optimizer = optim.Adam(vgg16.classifier.parameters(), lr=0.001)

In [ ]:
# Move model to GPU if available
vgg16 = vgg16.to(device)

for param in vgg16.parameters():
    param.data = param.data.to(device)
    if param.grad is not None:
        param.grad.data = param.grad.data.to(device)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    vgg16.train()  # Set model to training mode
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move to MPS

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = vgg16(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

    # Validation loop
    vgg16.eval()  # Set model to evaluation mode
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)  # Move to MPS
            outputs = vgg16(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Validation Loss: {val_loss/len(val_loader):.4f}, Accuracy: {100 * correct / total:.2f}%")

In [ ]:
torch.save(vgg16.state_dict(), '../vgg16_finetuned_pytorch.pth')

In [ ]:
# Load Model
# Load the model architecture
vgg16 = models.vgg16(pretrained=False)
vgg16.classifier = nn.Sequential(
    nn.Linear(25088, 4096),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5),
    nn.Linear(4096, 4096),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5),
    nn.Linear(4096, 10)  # Adjust for your number of classes
)

# Load the fine-tuned weights
vgg16.load_state_dict(torch.load('vgg16_finetuned.pth'))
vgg16 = vgg16.to(device)